In [ ]:
import pandas as pd

df=pd.read_pickle('/kaggle/input/building-strategyqa-sots-dataset/strategy-qa-sots-reasoning-path.pkl').to_pandas()
df.shape

In [ ]:
from typing import List
from relaxed_fda import RelaxedFDA
from prompt_template_collection import PromptTemplate
from doraemon import Doraemon

def context_to_str(context):
    if context is None:
        return ''
    return ' '.join(context)

logger = Doraemon.get_logger(name=__name__, logfile="relaxed_FDA_on_strategy_qa.log")

grouped = (df
    .groupby('question', sort=False)      # keep questions in first-seen order
    .agg(reasoning_paths=('reason', list), context=('evidence', 'first'), ground_truth=('ground_truth', 'first'))# collect each group's 'reason' values into a list
    .reset_index()                        # turn 'question' back into a column
)

grouped['context']=grouped['context'].apply(context_to_str)

In [ ]:
from typing import List, Dict, Tuple, Optional

task_name = "StrategyQA"

# few-shot examples with correct reasoning with wrong answer
D: List[Dict] = PromptTemplate.sot_construct_inter_strategyqa()

op_system_prompt = RelaxedFDA.get_optimize_system_prompt(task_name)
logger.info(op_system_prompt)

encoder=RelaxedFDA.get_encoder()

In [ ]:
result_pd: pd.DataFrame = RelaxedFDA.evaluate(
    grouped,
    logger,
    op_system_prompt,
    encoder,
    D,
    enable_logger_rs=False
)

In [ ]:
logger.info(f"Accuracy on {task_name} Dataset:{RelaxedFDA.calculate_accuracy(result_pd):.2%}")